https://www.analyticsvidhya.com/blog/2022/06/fine-tune-bert-model-for-named-entity-recognition-in-google-colab/

## Install Required Libraries

In [1]:
# We need to install the necessary libraries to work with the HuggingFace transformer
# datasets library to fetch data
# tokenizers to preprocess the data
# transformers to fine-tune the models
# seqeval to compute model metrics

!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q

## Load English dataset

We will be using an English language NER dataset from the HuggingFace datasets module. It follows the BIO (Beginning, Inside, Outside) format for tagging sentence tokens for the Named Entity Recognition task.

The dataset contains 3 sets of data, train, validation, and test. It consists of tokens, ner_tags, langs, and spans. The ner_tags have ids corresponding to BIO format, I-TYPE, which means the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have the tag B-TYPE to show that it starts a new phrase. A word with the tag O is not part of a phrase.

There is a total of 4 classes, Person(PER), Organization(ORG), Location(LOC), and others(O).

In [2]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "en")

/Users/owenmonroe/anaconda3/envs/textminingvis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset.keys()

dict_keys(['validation', 'test', 'train'])

In [4]:
label_names = dataset["train"].features["ner_tags"].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [5]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [6]:
dataset.column_names

{'validation': ['tokens', 'ner_tags', 'langs', 'spans'],
 'test': ['tokens', 'ner_tags', 'langs', 'spans'],
 'train': ['tokens', 'ner_tags', 'langs', 'spans']}

In [7]:
dataset.shape

{'validation': (10000, 4), 'test': (10000, 4), 'train': (20000, 4)}

In [8]:
dataset['train']

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans'],
    num_rows: 20000
})

In [9]:
dataset['train'][:2]

{'tokens': [['R.H.',
   'Saunders',
   '(',
   'St.',
   'Lawrence',
   'River',
   ')',
   '(',
   '968',
   'MW',
   ')'],
  [';', "'", "''", 'Anders', 'Lindström', "''", "'"]],
 'ner_tags': [[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0]],
 'langs': [['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'],
  ['en', 'en', 'en', 'en', 'en', 'en', 'en']],
 'spans': [['ORG: R.H. Saunders', 'ORG: St. Lawrence River'],
  ['PER: Anders Lindström']]}

## Data Preprocessing

- Bert expects input in `input_ids`, `token_type_ids` and `attention_mask` format
- The label also requires adjustment due to subword tokenization used by BERT

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

### Let's see why we need to adjust the labels

- We will process the tokens using tokenizer object

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding="max_length", truncation=True, is_split_into_words=True)

In [12]:
tokenized_datasets_ = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 20000/20000 [00:01<00:00, 11011.63 examples/s]


In [13]:
tokenized_datasets_['train'][0]['input_ids'][:20]

[101,
 1054,
 1012,
 1044,
 1012,
 15247,
 1006,
 2358,
 1012,
 5623,
 2314,
 1007,
 1006,
 5986,
 2620,
 12464,
 1007,
 102,
 0,
 0]

In [14]:
tokenized_datasets_['train'][0]['ner_tags'][:20]

[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]

In [15]:
len(tokenized_datasets_['train'][0]['input_ids']) == len(tokenized_datasets_['train'][0]['ner_tags'])

False

- We can see that len of `input_ids` is not matching with `ner_tags` that's why we require to adjust the labels according to the tokenized output

<hr/>

- We will use the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) as there is a sequence in data which has length>512

In [16]:
#Get the values for input_ids, attention_mask, adjusted labels
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True)

  total_adjusted_labels = []

  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []

    for word_idx in word_ids_list:
      # Special tokens have a word id that is None. We set the label to -100 so they are automatically
      # ignored in the loss function.
      if(word_idx is None):
        adjusted_label_ids.append(-100)
      elif(word_idx!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = word_idx
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)

  #add adjusted labels to the tokenized samples
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True, remove_columns=['tokens', 'ner_tags', 'langs', 'spans'])

Map: 100%|██████████| 20000/20000 [00:00<00:00, 30543.81 examples/s]


In [17]:
tokenized_dataset

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

- To understand word ids, consider following example

In [18]:
out = tokenizer("Fine tune NER in google colab!")
out

{'input_ids': [101, 2986, 8694, 11265, 2099, 1999, 8224, 15270, 2497, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
out.word_ids(0)

[None, 0, 1, 2, 2, 3, 4, 5, 5, 6, None]

Here, we can see 2 and 5 ids are repeated twice due to sub-word tokenization



- We will now have all the required fields for training, 'input_ids', 'token_type_ids', 'attention_mask', 'labels'

In [20]:
tokenized_dataset

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

In [21]:
tokenized_dataset['train'][:2]

{'input_ids': [[101,
   1054,
   1012,
   1044,
   1012,
   15247,
   1006,
   2358,
   1012,
   5623,
   2314,
   1007,
   1006,
   5986,
   2620,
   12464,
   1007,
   102],
  [101,
   1025,
   1005,
   1005,
   1005,
   15387,
   11409,
   5104,
   13887,
   1005,
   1005,
   1005,
   102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[-100, 3, 3, 3, 3, 4, 0, 3, 3, 4, 4, 0, 0, 0, 0, 0, 0, -100],
  [-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, -100]]}

- As we can see, different sample have different length therefore we need to
pad the tokens to have same length

- https://huggingface.co/docs/transformers/main/main_classes/data_collator#transformers.DataCollatorForTokenClassification

In [22]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
data_collator

DataCollatorForTokenClassification(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_tok

## Fine Tuning

In [24]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, AdamW

In [25]:
#check if gpu is present
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

- We will use Distillbert-base-uncased model for fine tuning
- We need to specify the number of labels present in the dataset

In [26]:
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_names))
model.to(device)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

- Create a function to generate metrics
- We will use `seqeval` metrics, commonly used for token classification

In [27]:
!pip install seqeval -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p

    #select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_6544/4044278287.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [29]:
example = dataset["train"][1]
labels = [label_names[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

- Fine Tuning using Trainer API

In [30]:
! pip install -U accelerate
! pip install -U transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
from transformers import TrainingArguments, Trainer

batch_size = 16
logging_steps = len(tokenized_dataset['train']) // batch_size
epochs = 2

training_args = TrainingArguments(
    output_dir="Lab11_Nov13_NER/NER-BERT-colab",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps)

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [48]:
trainer.train_dataset[0]

{'input_ids': [101,
  1054,
  1012,
  1044,
  1012,
  15247,
  1006,
  2358,
  1012,
  5623,
  2314,
  1007,
  1006,
  5986,
  2620,
  12464,
  1007,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 3, 3, 3, 4, 0, 3, 3, 4, 4, 0, 0, 0, 0, 0, 0, -100]}

In [49]:
trainer.eval_dataset[0]

{'input_ids': [101,
  16615,
  4212,
  5196,
  1006,
  16615,
  4212,
  1010,
  2148,
  7734,
  1007,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 4, 4, 0, 5, 6, 6, 6, 6, 0, -100]}

In [50]:
#fine tune using train method
trainer.train()

 33%|███▎      | 832/2500 [2:05:51<08:44,  3.18it/s]     

In [ ]:
trainer.evaluate()

To get the precision/recall/f1 computed for each category for test set, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels)
results

## Observations

- f1 score for LOC and PER is >85% and ORG has <75%
- Overall f1 score is ~83%
- We can improve the accuracy by training the model for more number of epochs (currently only 2 epochs)